In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
cd ..

C:\Projects\python\recommender


In [17]:
from pathlib import Path

import numpy as np
import scipy.sparse as sp
import pandas as pd
import torch as T

from datasets import TorchTopcoder

In [4]:
DEVICE = T.device('cpu')
BATCH = 3
SHUFFLE = True
WORKER_NUM = 0
REGS_PATH = Path("./inputs/topcoder/regs.csv")
CHAG_PATH = Path("./inputs/topcoder/challenge.csv")

In [5]:
databunch = TorchTopcoder(regs_path=REGS_PATH, chag_path=CHAG_PATH)
databunch

2019-09-10 09:15:04,969 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
2019-09-10 09:15:04,969 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
I0910 09:15:04.969359  7596 torch_topcoder.py:68] Read dataset in inputs\topcoder\regs.csv
2019-09-10 09:15:04,973 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
2019-09-10 09:15:04,973 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
I0910 09:15:04.973351  7596 torch_topcoder.py:69] Original regs shape: (610025, 3)
2019-09-10 09:15:05,069 - C:\Projects\python\recommender\utils.py - INFO - Original registants size: 60017
2019-09-10 09:15:05,069 - C:\Projects\python\recommender\utils.py - INFO - Original registants size: 60017
I0910 09:15:05.069129  7596 torch_topcoder.py:74] Original registants size: 60017
2019-09-10 09:15:05,073 - C:\Projects\python\recommender\utils.py - INFO - Origina

In [11]:
databunch.feat_dim

69384

In [12]:
batch_df = databunch._df_dict['train'].sample(10)
batch_df

,registant,challengeId,previousId
417960,westine,30025697,30018643
553644,wang_liang,30006898,30006472
302707,ahmed.seddiq,30042394,30039533
371797,rtriv,30032197,30032201
184062,genycopedison,30055866,30055861
414283,saarixx,30026198,30024075
466059,arylio,30019790,30023300
185271,tototpc,30055936,30055716
181787,blu3fox,30055732,30055725
494704,winterflame,30015688,30015841


In [15]:
chag_encoder = databunch._chag_encoder
regs_encoder = databunch._regs_encoder
chag_encoder.categories_

[array([30000030, 30000070, 30000071, ..., 30072066, 30072067, 30072069],
       dtype=int64)]

In [19]:
batch_df['negtiveId'] = np.random.choice(a=chag_encoder.categories_[0], size=10)
batch_df

,registant,challengeId,previousId,negtiveId
417960,westine,30025697,30018643,30047815
553644,wang_liang,30006898,30006472,30003021
302707,ahmed.seddiq,30042394,30039533,30027354
371797,rtriv,30032197,30032201,30019225
184062,genycopedison,30055866,30055861,30045622
414283,saarixx,30026198,30024075,30062152
466059,arylio,30019790,30023300,30051111
185271,tototpc,30055936,30055716,30031354
181787,blu3fox,30055732,30055725,30054681
494704,winterflame,30015688,30015841,30042361


In [20]:
regs_vector: sp.csr_matrix = regs_encoder.transform(batch_df[['registant']])
chag_vector = chag_encoder.transform(batch_df[['challengeId']])
prev_vector = chag_encoder.transform(batch_df[['previousId']])
negi_vector = chag_encoder.transform(batch_df[['negtiveId']])

In [21]:
pos_feat_matrix = sp.hstack([regs_vector, prev_vector, chag_vector])
neg_feat_matrix = sp.hstack([regs_vector, prev_vector, negi_vector])

In [22]:
print(pos_feat_matrix.shape)
print(neg_feat_matrix.shape)

(10, 68781)
(10, 68781)


In [26]:
print(regs_vector.shape)
print(regs_encoder.categories_[0].size)

(10, 9829)
9829
